In [1]:
import os
import pandas as pd
import math
from prettytable import PrettyTable
from operator import itemgetter

## Computing number of samples as a function of epsilon and the other way around

In [2]:
def compute_n(c, delta, epsilon, size_of_market):
    return (c / epsilon) * (c / epsilon) * 0.5 * math.log((2 * size_of_market) / delta)

In [3]:
def compute_eps(c, delta, size_of_market, number_of_samples):
    return c * math.sqrt((math.log((2.0 * size_of_market) / delta)) / (2.0 * number_of_samples))

### Testing eps to n and n to eps functions

In [4]:
c = 2
delta = 0.1
epsilon = 0.05
size_of_market = 4224
n = compute_n(c, delta, epsilon, size_of_market)
assert epsilon == compute_eps(c, delta, size_of_market, n)
assert n == compute_n(c, delta, epsilon, size_of_market)

## Compute Statistics from LSVM worlds

In [5]:
results_dir = '../LSVM/worlds_results/'

In [6]:
def compare_ea_eap(world_number):
    # Read EAP data.
    bidders_final_values_df = pd.read_csv(f"{results_dir}word{world_number}/bidders_final_values.csv")
    eap_samples = bidders_final_values_df['n'].sum()
    
    # Compute EA data.
    bidders_summary_df = pd.read_csv(f"{results_dir}word{world_number}/bidders_summary.csv")
    params_csv = pd.read_csv(f"{results_dir}word{world_number}/params.csv")
    c, delta, epsilon = int(params_csv.iloc[3]['value']), sum(eval(params_csv.iloc[1]['value'])), float(params_csv.iloc[6]['value'])
    size_of_market = sum([2 ** len(eval(row[2])) for row in bidders_summary_df.itertuples()])
    ea_samples = compute_n(c, delta, epsilon, size_of_market) * size_of_market
    #print(f"c = {c}, delta = {delta}, epsilon = {epsilon : .4f}, size_of_market = {size_of_market}")
    #print(f"{ ea_samples : ,.0f}")
    #print(f"{ eap_samples : ,}")
    
    return [world_number, 
            c, 
            delta, 
            f"{epsilon : .4f}", 
            f"{size_of_market: ,}", 
            f"{ea_samples : ,.0f}", 
            f"{eap_samples : ,.0f}",
            f"{(eap_samples / ea_samples) * 100.0 : .4f}"]

In [7]:
results_ptable = PrettyTable()
results_ptable.field_names = ['world', 'c', 'delta', 'epsilon', 'size_of_market', 'ea', 'eap', 'ratio']
results_ptable.align["ea"] = "r"
results_ptable.align["eap"] = "r"
results = []
for i in range(0, 100):
    results += [compare_ea_eap(i)]
results = sorted(results, key=itemgetter(7))

for row in results:
    results_ptable.add_row(row)
print(results_ptable)

+-------+---+-------+---------+----------------+--------------+----------+---------+
| world | c | delta | epsilon | size_of_market |           ea |      eap |  ratio  |
+-------+---+-------+---------+----------------+--------------+----------+---------+
|   67  | 2 |  0.1  |  0.0346 |      7,424     |  147,554,103 |  125,270 |  0.0849 |
|   86  | 2 |  0.1  |  0.0346 |      5,440     |  105,298,345 |  104,890 |  0.0996 |
|   37  | 2 |  0.1  |  0.0346 |      5,120     |   98,586,258 |  102,230 |  0.1037 |
|   5   | 2 |  0.1  |  0.0346 |      5,440     |  105,298,345 |  110,020 |  0.1045 |
|   74  | 2 |  0.1  |  0.0346 |      5,312     |  102,609,633 |  107,210 |  0.1045 |
|   65  | 2 |  0.1  |  0.0346 |      4,736     |   90,576,036 |   98,480 |  0.1087 |
|   70  | 2 |  0.1  |  0.0346 |      4,608     |   87,917,311 |  100,980 |  0.1149 |
|   85  | 2 |  0.1  |  0.0346 |      4,800     |   91,907,573 |  105,690 |  0.1150 |
|   14  | 2 |  0.1  |  0.0346 |      4,416     |   83,940,404 |  